In [2]:
import pandas as pd
from typing import List, Dict, Union

class ExcelMatcher:
    """
    Excel文件匹配工具类
    功能：对比两个Excel文件的指定字段，生成匹配报告
    
    使用示例：
    matcher = ExcelMatcher()
    matcher.run_match(
        "A.xlsx",
        "B.xlsx",
        ["Name", "ID", "Amount"],
        ["Customer", "RefID", "Value"],
        "匹配报告.xlsx"
    )
    """
    
    def __init__(self):
        self.df_a = None
        self.df_b = None
        self.a_columns = []
        self.b_columns = []
        
    def _load_data(self, file_a: str, file_b: str) -> None:
        """加载Excel文件"""
        self.df_a = pd.read_excel(file_a).convert_dtypes()
        self.df_b = pd.read_excel(file_b).convert_dtypes()
        
        # 添加原始行号（兼容Excel行号，假设首行为标题）
        self.df_a["__A_Index"] = self.df_a.index + 2
        self.df_b["__B_Index"] = self.df_b.index + 2
    
    def _validate_columns(self, a_cols: List[str], b_cols: List[str]) -> None:
        """验证字段是否存在"""
        for col in a_cols:
            if col not in self.df_a.columns:
                raise ValueError(f"A文件中不存在字段: {col}")
        for col in b_cols:
            if col not in self.df_b.columns:
                raise ValueError(f"B文件中不存在字段: {col}")
    
    def _exact_match(self) -> Dict[str, pd.DataFrame]:
        """执行精确匹配"""
        merged = self.df_a.merge(
            self.df_b,
            left_on=self.a_columns,
            right_on=self.b_columns,
            how="outer",
            indicator=True
        )
        
        return {
            "perfect": merged[merged["_merge"] == "both"],
            "only_a": merged[merged["_merge"] == "left_only"],
            "only_b": merged[merged["_merge"] == "right_only"]
        }
    
    def _generate_mismatch_detail(self, df: pd.DataFrame, source: str) -> pd.Series:
        """生成不匹配字段详情"""
        def _tag_row(row):
            missing = []
            for a_col, b_col in zip(self.a_columns, self.b_columns):
                if source == "A" and pd.isna(row[b_col]):
                    missing.append(f"A文件第{row['__A_Index']}行.{a_col}={row[a_col]}")
                elif source == "B" and pd.isna(row[a_col]):
                    missing.append(f"B文件第{row['__B_Index']}行.{b_col}={row[b_col]}")
            return " | ".join(missing) if missing else "完全匹配"
        
        return df.apply(_tag_row, axis=1)
    
    def _generate_summary(self, result: Dict[str, pd.DataFrame]) -> pd.DataFrame:
        """生成匹配统计"""
        return pd.DataFrame([{
            "总匹配数": len(result["perfect"]),
            "A表记录数": len(self.df_a),
            "B表记录数": len(self.df_b),
            "A独有记录": len(result["only_a"]),
            "B独有记录": len(result["only_b"]),
            "匹配率": f"{len(result['perfect'])/max(len(self.df_a), len(self.df_b)):.2%}"
        }])
    
    def run_match(
        self,
        file_a: str,
        file_b: str,
        a_columns: List[str],
        b_columns: List[str],
        output_file: str = "匹配报告.xlsx"
    ) -> Dict[str, Union[int, pd.DataFrame]]:
        """
        执行匹配流程
        
        参数：
        file_a: A文件路径
        file_b: B文件路径
        a_columns: A文件需要匹配的字段列表
        b_columns: B文件对应字段列表（顺序需与a_columns一致）
        output_file: 输出报告路径
        
        返回：
        包含统计信息和DataFrame的字典
        """
        # 1. 初始化配置
        self.a_columns = a_columns
        self.b_columns = b_columns
        
        # 2. 加载数据
        self._load_data(file_a, file_b)
        self._validate_columns(a_columns, b_columns)
        
        # 3. 执行匹配
        match_result = self._exact_match()
        
        # 4. 处理匹配结果
        match_result["only_a"]["Mismatch_Detail"] = self._generate_mismatch_detail(
            match_result["only_a"], "A"
        )
        match_result["only_b"]["Mismatch_Detail"] = self._generate_mismatch_detail(
            match_result["only_b"], "B"
        )
        
        # 5. 生成报告
        summary = self._generate_summary(match_result)
        
        with pd.ExcelWriter(output_file) as writer:
            summary.to_excel(writer, sheet_name="匹配统计", index=False)
            match_result["perfect"][a_columns + b_columns].to_excel(
                writer, sheet_name="匹配成功", index=False
            )
            fail_report = pd.concat([
                match_result["only_a"][["__A_Index", *a_columns, "Mismatch_Detail"]],
                match_result["only_b"][["__B_Index", *b_columns, "Mismatch_Detail"]]
            ])
            fail_report.to_excel(writer, sheet_name="匹配失败", index=False)
        
        print(f"匹配完成！结果已保存到 {output_file}")
        
        return {
            "summary": summary.iloc[0].to_dict(),
            "perfect_matches": match_result["perfect"],
            "failed_matches": fail_report
        }



In [3]:
# ==================== 使用示例 ====================
if __name__ == "__main__":
    matcher = ExcelMatcher()
    result = matcher.run_match(
        file_a="A.xlsx",
        file_b="B.xlsx",
        a_columns=["Name", "ID", "Amount"],
        b_columns=["Customer", "RefID", "Value"],
        output_file="高级匹配报告.xlsx"
    )
    
    # 访问结果数据
    print("\n匹配统计:")
    print(result["summary"])
    print("\n前5条匹配失败记录:")
    print(result["failed_matches"].head())

匹配完成！结果已保存到 高级匹配报告.xlsx

匹配统计:
{'总匹配数': 1, 'A表记录数': 4, 'B表记录数': 4, 'A独有记录': 3, 'B独有记录': 3, '匹配率': '25.00%'}

前5条匹配失败记录:
   __A_Index    Name    ID  Amount  \
0        5.0   huliu   105      33   
1        2.0    lisi    99      80   
5        4.0  wangwu   101      82   
2        NaN    <NA>  <NA>    <NA>   
3        NaN    <NA>  <NA>    <NA>   

                                     Mismatch_Detail  __B_Index Customer  \
0  A文件第5.0行.Name=huliu | A文件第5.0行.ID=105 | A文件第5....        NaN     <NA>   
1  A文件第2.0行.Name=lisi | A文件第2.0行.ID=99 | A文件第2.0行...        NaN     <NA>   
5  A文件第4.0行.Name=wangwu | A文件第4.0行.ID=101 | A文件第4...        NaN     <NA>   
2  B文件第4.0行.Customer=lisi | B文件第4.0行.RefID=102 | ...        4.0     lisi   
3  B文件第5.0行.Customer=tianqi | B文件第5.0行.RefID=103 ...        5.0   tianqi   

   RefID  Value  
0   <NA>   <NA>  
1   <NA>   <NA>  
5   <NA>   <NA>  
2    102     80  
3    103     80  
